In [43]:
import numpy as np
import pandas
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix,roc_curve,auc,precision_recall_curve,precision_score,recall_score,f1_score,accuracy_score

#from pymystem3 import Mystem
import pymorphy2


In [ ]:
# инициализируем
vectorizer = CountVectorizer()
#m = Mystem()
morph = pymorphy2.MorphAnalyzer()

In [26]:
# функция для лемматизации

def lemmatize(text):
    text_str = str(text)
    text_str = re.sub(r'[^\w\s]+|[\d]+', r'',text_str).strip()
    words = text_str.split() # разбиваем текст на слова
    res = ""
    for word in words:
        p = morph.parse(word)[0]
        res = res + " " + p.normal_form

    return res

In [27]:
data_xls = pandas.read_excel("resolution_100.xlsx")

In [28]:
# проводим лемматизацию
data_xls['text'] = data_xls.apply(lambda row: lemmatize(row['text']), axis=1)

In [29]:
text = data_xls.text

In [30]:
# создаем векторы
text_vector = vectorizer.fit_transform(text)

In [31]:
# показать матрицу
text_vector_array = text_vector.toarray()

In [32]:
past_columns = [] # пустой массив, для опрееделени колонок
for i in range(len(text_vector_array[0])): 
    past_columns.append(f"v_{i}")
#past_columns

In [33]:
dataframe_vector = pandas.DataFrame(data = text_vector_array,columns = past_columns)
#dataframe_vector

In [34]:
data = pandas.concat([data_xls,dataframe_vector],axis = 1)


In [35]:
Y = data.priznak
X = data[past_columns]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

In [39]:
# предсказание простое
clf = RandomForestClassifier(random_state=241)
clf.fit(X_train, y_train)
# расчет веса показателей
importances = clf.feature_importances_
print(X.columns)
print(importances)

Index(['v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9',
       ...
       'v_1326', 'v_1327', 'v_1328', 'v_1329', 'v_1330', 'v_1331', 'v_1332',
       'v_1333', 'v_1334', 'v_1335'],
      dtype='object', length=1336)
[3.58701115e-04 1.63065019e-06 4.24769699e-04 ... 4.13800834e-04
 1.78208538e-05 1.91690868e-07]


In [41]:
y_pred=clf.predict(X_test)

In [44]:
accuracy_score(y_test,y_pred)

0.95

In [38]:
# ДЛЯ ПРЯМОГО РАСЧЕТА
# теперь можно быстро подсчитать вектор для нового документа
#text2 = ["the puppy"]
#vector = vectorizer.transform(text2)
#print(vector.toarray())

text_predict = 'Подготовить отчет'
text_predict_lemmat = lemmatize(text_predict)
new_v = vectorizer.transform([text_predict_lemmat])  # результат [[1 0 0 2]]
new_v_array = new_v.toarray()
dataframe_predict = pandas.DataFrame(data = new_v_array,columns = past_columns)
print(dataframe_predict)
clf.predict(dataframe_predict)

   v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  ...  v_1326  v_1327  \
0    0    0    0    0    0    0    0    0    0    0  ...       0       0   

   v_1328  v_1329  v_1330  v_1331  v_1332  v_1333  v_1334  v_1335  
0       0       0       0       0       0       0       0       0  

[1 rows x 1336 columns]


array([1])

In [117]:
# СОХРАНЕНИЕ МОДЕЛИ
filename = 'finalized_model_resolut.sav'
pickle.dump(clf, open(filename, 'wb'))

In [25]:
# СОХРАНЕНИЕ ВЕКТОРОВ
filename = 'finalized_vector_resolut.sav'
pickle.dump(vectorizer, open(filename, 'wb'))